# Configs

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
DATA_DIR = 'papers/'
VECTOR_STORE_PATH = 'chroma_db/'

# Ingest PDFs

In [2]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [3]:
def ingest_pdfs():
    documents = []
    for filename in os.listdir(DATA_DIR):
        if filename.endswith(".pdf"):
            loader = PyPDFLoader(os.path.join(DATA_DIR, filename))
            documents.extend(loader.load())

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(documents)

    embeddings = OpenAIEmbeddings()
    Chroma.from_documents(splits, embeddings, persist_directory=VECTOR_STORE_PATH)
    print("PDFs ingested successfully.")

In [5]:
ingest_pdfs()

PDFs ingested successfully.


# Memory

In [6]:
from collections import defaultdict

class MemoryManager:
    def __init__(self):
        self.sessions = defaultdict(list)

    def add_message(self, session_id: str, message: dict):
        self.sessions[session_id].append(message)

    def get_session_history(self, session_id: str) -> list:
        return self.sessions.get(session_id, [])

    def clear_session(self, session_id: str):
        if session_id in self.sessions:
            del self.sessions[session_id]

In [7]:
memory = MemoryManager()

# Agents

## Clarification

In [8]:
from langchain_openai import ChatOpenAI

def clarify_query(query: str, session_id: str, llm: ChatOpenAI, memory: MemoryManager) -> str:
    history = memory.get_session_history(session_id)
    prompt = f"""
    Conversation history: {history}
    Analyze the query: "{query}"
    If it is vague or underspecified (e.g., missing context or unclear terms), return a clarified version or a question by looking at the conversation history to ask the chatbot.
    If it is clear, return the original query.
    Example: hisory: "OCR-Free Document Understanding Transformer" paper 
             query: "What is the accuracy?"
             clarified_query: "What is the accuracy of the model in the 'OCR-Free Document Understanding Transformer' paper?"
    """
    response = llm.invoke(prompt)
    return response.content

# Routing

In [9]:
from langchain_openai import ChatOpenAI

def route_query(query: str, llm: ChatOpenAI) -> str:
    prompt = f"""
    Determine if the query "{query}" can be answered using academic papers or requires a web search.
    Return "pdf" for queries related to academic papers, or "web" for queries requiring external information.
    Example: "What did OpenAI release this month?" -> "web"
    Example: "Which prompt template gave the highest zero-shot accuracy in Zhang et al.?" -> "pdf"
    """
    response = llm.invoke(prompt)
    return response.content

# Retrieval

In [10]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma  
from langchain.prompts import PromptTemplate

def retrieve_and_answer(query: str, llm: ChatOpenAI) -> str:
    vector_store = Chroma(persist_directory=VECTOR_STORE_PATH, embedding_function=OpenAIEmbeddings())
    retriever = vector_store.as_retriever(search_kwargs={"k": 5})
    docs = retriever.invoke(query) 

    prompt = PromptTemplate(
        input_variables=["query", "context"],
        template="Based on the context: {context}\nAnswer the query: {query}"
    )
    context = "\n".join([doc.page_content for doc in docs])
    response = llm.invoke(prompt.format(query=query, context=context))
    return response.content

# Web Search

In [11]:
from tavily import TavilyClient
from langchain_openai import ChatOpenAI

def web_search(query: str, llm: ChatOpenAI, tavily: TavilyClient) -> str:
    results = tavily.search(query=query, max_results=3)
    context = "\n".join([result["content"] for result in results["results"]])
    prompt = f"Based on the web search results: {context}\nAnswer the query: {query}"
    response = llm.invoke(prompt)
    return response.content

# Generation

In [12]:
from langchain_openai import ChatOpenAI

def generate_answer(query: str, context: str, session_id: str, llm: ChatOpenAI, memory: MemoryManager) -> str:
    history = memory.get_session_history(session_id)
    prompt = f"""
    Conversation history: {history}
    Context: {context}
    Query: {query}
    Provide a concise and accurate answer.
    """
    response = llm.invoke(prompt)
    memory.add_message(session_id, {"query": query, "answer": response.content})
    return response.content

# Data Models

In [13]:
from typing import Dict, Any
from pydantic import BaseModel

In [44]:
class QueryRequest(BaseModel):
    query: str
    session_id: str
    web_search: bool

class State(Dict[str, Any]):
    query: str
    session_id: str
    clarified_query: str
    route_decision: str
    context: str
    response: str
    web_search: bool

# Lang Graph

In [45]:
from langgraph.graph import StateGraph, END

In [46]:
llm = ChatOpenAI(model="gpt-4o-mini")
tavily = TavilyClient(api_key=TAVILY_API_KEY)

In [47]:
def clarification_node(state: State) -> State:
    state["clarified_query"] = clarify_query(state["query"], state["session_id"], llm, memory)
    return state

def routing_node(state: State) -> State:
    if state["web_search"]:
        state["route_decision"] = 'web'
    else:
        state["route_decision"] = route_query(state["clarified_query"], llm)
    return state

def retrieval_node(state: State) -> State:
    if state["route_decision"] == "pdf":
        state["context"] = retrieve_and_answer(state["clarified_query"], llm)
    else:
        state["context"] = web_search(state["clarified_query"], llm, tavily)
    return state

def answer_node(state: State) -> State:
    state["response"] = generate_answer(state["clarified_query"], state["context"], state["session_id"], llm, memory)
    return state

In [48]:
workflow = StateGraph(State)
workflow.add_node("clarify", clarification_node)
workflow.add_node("route", routing_node)
workflow.add_node("retrieve", retrieval_node)
workflow.add_node("answer", answer_node)
workflow.add_edge("clarify", "route")
workflow.add_edge("route", "retrieve")
workflow.add_edge("retrieve", "answer")
workflow.add_edge("answer", END)
workflow.set_entry_point("clarify")
graph = workflow.compile()

# Testing

In [49]:
memory.clear_session("user1")

In [16]:
state = graph.invoke({"query": "Do you know about 'How to Prompt LLMs for Text-to-SQL' paper?", "session_id": "user1", "web_search": False})
print(state['response'])

The paper "How to Prompt LLMs for Text-to-SQL" presents several key findings and contributions:

1. **Comparative Analysis of Prompt Constructions**: It highlights the variability in prompt strategies from previous research and their impact on LLM performance in text-to-SQL tasks.

2. **Optimal Prompt Template Identification**: The authors identify the best prompt template by partitioning prompts into components and testing their effectiveness in enhancing LLM performance.

3. **Benchmarking Across Various LLMs**: The research benchmarks a range of LLMs, including general-purpose and coding-specific models, to evaluate performance differences and boundaries.

4. **Impact of Information Granularity**: It assesses how the level of detail in prompts affects model performance and identifies optimal learning strategies, such as zero-shot and few-shot approaches.

Overall, the paper aims to improve the generation of SQL queries by optimizing prompt strategies and understanding their influenc

In [17]:
state

{'query': "Do you know about 'How to Prompt LLMs for Text-to-SQL' paper?",
 'session_id': 'user1',
 'clarified_query': 'clarified_query: "What are the main findings or contributions of the paper \'How to Prompt LLMs for Text-to-SQL\'?"',
 'route_decision': 'pdf',
 'context': 'The paper "How to Prompt LLMs for Text-to-SQL" presents several key findings and contributions:\n\n1. **Comparative Analysis of Prompt Constructions**: The study investigates various prompt strategies employed in previous research, highlighting the lack of comparability in prompt constructions and their contributions to LLM performance in text-to-SQL tasks.\n\n2. **Optimal Prompt Template Identification**: The authors partition prompt text into distinct components and conduct thorough testing to determine the optimal prompt template for enhancing LLM performance on end-to-end text-to-SQL tasks.\n\n3. **Benchmarking Across Various LLMs**: The research includes a benchmarking approach that evaluates a range of LLMs,

In [18]:
state = graph.invoke({"query": "What metrics are used to measure the SQL quality in that paper?", "session_id": "user1", "web_search": False})
print(state['response'])

The paper "How to Prompt LLMs for Text-to-SQL" uses the following metrics to measure the quality of SQL queries:

1. **F1 values of Rouge-1/2/L** - Evaluates the overlap between generated SQL and expected SQL based on n-grams.
2. **BertScore** - Assesses the quality of generated SQL by comparing it to reference texts using contextual embeddings.
3. **Semantic coherence assessment** - Utilizes LLMs to evaluate the semantic coherence between generated SQL statements and original natural language questions.

These metrics aim to assess both the accuracy and semantic understanding of the generated SQL queries.


In [19]:
state

{'query': 'What metrics are used to measure the SQL quality in that paper?',
 'session_id': 'user1',
 'clarified_query': 'clarified_query: "What metrics are used to measure the SQL quality in the paper \'How to Prompt LLMs for Text-to-SQL\'?"',
 'route_decision': 'pdf',
 'context': 'The paper "How to Prompt LLMs for Text-to-SQL" uses several metrics to measure the quality of SQL queries. These metrics include:\n\n1. **F1 values of Rouge-1/2/L** - These metrics evaluate the overlap between the generated SQL query and the expected SQL query in terms of n-grams.\n\n2. **BertScore** - This metric assesses the quality of generated text by comparing it to reference texts using contextual embeddings.\n\n3. **Semantic coherence assessment** - The paper also utilizes LLMs to evaluate the semantic coherence between the generated SQL statements and the original natural language questions.\n\nThese evaluation metrics are designed to assess the accuracy and semantic understanding of the SQL generat

In [20]:
state = graph.invoke({"query": "Is there any github code for that paper?", "session_id": "user1", "web_search": False})
print(state['response'])

Yes, there is GitHub code available for the paper "How to Prompt LLMs for Text-to-SQL." You can find the repository [here](https://github.com/shuaichenchang/prompt-text-to-sql).


In [21]:
state

{'query': 'Is there any github code for that paper?',
 'session_id': 'user1',
 'clarified_query': 'clarified_query: "Is there any GitHub code available for the paper \'How to Prompt LLMs for Text-to-SQL\'?"',
 'route_decision': 'web',
 'context': 'Yes, there is GitHub code available for the paper "How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings." You can find the repository [here](https://github.com/shuaichenchang/prompt-text-to-sql). The repository includes Python scripts like `text_to_sql.py`, which you can use to run text-to-SQL tasks with various settings and models. For example, you can run it with the Codex model in a zero-shot setting using the command:\n\n```bash\npython text_to_sql.py --setting zeroshot --model codex --prompt_db "CreateTableSelectCol"\n```',
 'response': 'Yes, there is GitHub code available for the paper "How to Prompt LLMs for Text-to-SQL." You can find the repository [here](https://github.com/shuaichenchang/

In [16]:
state = graph.invoke({"query": "How many examples are enough for good accuracy", "session_id": "user1", "web_search": False})
print(state['response'])

In general, a few hundred examples are often considered sufficient to achieve good accuracy in many machine learning tasks, though this can vary based on factors such as task complexity, data diversity, and model architecture. In few-shot learning scenarios, even 5 to 10 examples can improve performance for adaptable models like Codex. However, there is a threshold beyond which additional examples may lead to diminishing returns.


In [17]:
state

{'query': 'How many examples are enough for good accuracy',
 'session_id': 'user1',
 'clarified_query': 'clarified_query: "In the context of training a machine learning model, how many examples are generally considered enough to achieve good accuracy?"',
 'route_decision': 'pdf',
 'context': 'In the context of training a machine learning model, the number of examples considered enough to achieve good accuracy can vary significantly based on several factors, including the complexity of the task, the diversity of the data, and the specific model architecture being used. \n\nFrom the information provided, we can see examples from the Scholar and GeoQuery datasets. The finetuned T5 model achieved 87.2% accuracy with 499 examples for Scholar and 85.7% accuracy with 549 examples for GeoQuery. These figures suggest that a few hundred examples can yield competitive accuracy for certain tasks, particularly in the context of text-to-SQL systems.\n\nIn few-shot learning scenarios, it appears that

In [18]:
state = graph.invoke({"query": "Which prompt template gave the highest zero-shot accuracy on Spider in Zhang et al. (2024)?", "session_id": "user1", "web_search": False})
print(state['response'])

The prompt template that yielded the highest zero-shot accuracy on the Spider dataset according to the study by Zhang et al. (2024) is "Create Table + Select 3."


In [19]:
state

{'query': 'Which prompt template gave the highest zero-shot accuracy on Spider in Zhang et al. (2024)?',
 'session_id': 'user1',
 'clarified_query': 'clarified_query: "Which prompt template yielded the highest zero-shot accuracy on the Spider dataset according to the study by Zhang et al. (2024)?"',
 'route_decision': 'pdf',
 'context': 'Based on the information provided, the highest zero-shot accuracy on the Spider dataset according to the study is achieved by the prompt template "Create Table + Select 3." This combination yielded the best results in terms of valid SQL predictions, execution accuracy, and test-suite accuracy for the Codex model.',
 'response': 'The prompt template that yielded the highest zero-shot accuracy on the Spider dataset according to the study by Zhang et al. (2024) is "Create Table + Select 3."'}

In [20]:
state = graph.invoke({"query": "What execution accuracy does davinci-codex reach on Spider with the ‘Create Table + Select 3’ prompt?", "session_id": "user1", "web_search": False})
print(state['response'])

The Davinci Codex model achieves an execution accuracy of 67.0% on the Spider dataset using the "Create Table + Select 3" prompt.


In [21]:
state

{'query': 'What execution accuracy does davinci-codex reach on Spider with the ‘Create Table + Select 3’ prompt?',
 'session_id': 'user1',
 'clarified_query': 'clarified_query: "What execution accuracy does the Davinci Codex model achieve on the Spider dataset using the \'Create Table + Select 3\' prompt?"',
 'route_decision': 'pdf',
 'context': 'The Davinci Codex model achieves an execution accuracy of 67.0% on the Spider dataset using the "Create Table + Select 3" prompt.',
 'response': 'The Davinci Codex model achieves an execution accuracy of 67.0% on the Spider dataset using the "Create Table + Select 3" prompt.'}

In [22]:
state = graph.invoke({"query": "What did OpenAI release this month?", "session_id": "user1", "web_search": False})
print(state['response'])

In October 2023, OpenAI announced the upcoming release of GPT-5 and discussed the new ChatGPT Agent feature, indicating that user feedback from these agents may contribute to the model's training. Additionally, they have been gradually rolling out long-term memory in ChatGPT to enhance its ability to remember user information.


In [23]:
state

{'query': 'What did OpenAI release this month?',
 'session_id': 'user1',
 'clarified_query': 'clarified_query: "What specific release or announcement did OpenAI make this month?"',
 'route_decision': 'web',
 'context': 'In October 2023, OpenAI made significant announcements regarding the development of GPT-5, as mentioned by Xikun Zhang during a discussion of the new ChatGPT Agent feature. This announcement indicated that GPT-5 "is coming," and there are expectations that user feedback from ChatGPT Agents may contribute to its training. Additionally, OpenAI has been gradually rolling out long-term memory in ChatGPT, enhancing its ability to remember user information, which might be further improved with the release of GPT-5.',
 'response': "In October 2023, OpenAI announced the upcoming release of GPT-5 and discussed the new ChatGPT Agent feature, indicating that user feedback from these agents may contribute to the model's training. Additionally, they have been gradually rolling out l

In [51]:
state = graph.invoke({"query": "What did OpenAI release in 2025?", "session_id": "user1", "web_search": True})
print(state['response'])

OpenAI is expected to release GPT-5 and a new open-weights model in August 2025.


In [52]:
state

{'query': 'What did OpenAI release in 2025?',
 'session_id': 'user1',
 'clarified_query': 'clarified_query: "What specific product or technology did OpenAI release in 2025?"',
 'route_decision': 'web',
 'context': 'Based on the information provided, OpenAI is expected to release its next big model, GPT-5, in August 2025. Additionally, there is mention of a new open-weights model that could also be released around the same time, although the exact timing may vary. This would mark the first open-weights model release from OpenAI since GPT-2 in 2019.',
 'response': 'OpenAI is expected to release GPT-5 and a new open-weights model in August 2025.',
 'web_search': True}